## 주식 예측

https://finance.yahoo.com/quote/005930.KS/history?period1=1441065600&period2=1598918400&interval=1d&filter=history&frequency=1d

에서 5년치 주식 다운로드

In [1]:
import pandas as pd

data = pd.read_csv('./stock/AMZN.csv')

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1259 entries, 0 to 1258
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       1259 non-null   object 
 1   Open       1259 non-null   float64
 2   High       1259 non-null   float64
 3   Low        1259 non-null   float64
 4   Close      1259 non-null   float64
 5   Adj Close  1259 non-null   float64
 6   Volume     1259 non-null   int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 69.0+ KB


In [2]:
data.info

<bound method DataFrame.info of             Date         Open         High          Low        Close  \
0     2015-09-01   499.140015   510.000000   493.429993   496.540009   
1     2015-09-02   505.089996   510.859985   497.720001   510.549988   
2     2015-09-03   514.500000   515.840027   502.570007   504.720001   
3     2015-09-04   497.649994   502.850006   495.640015   499.000000   
4     2015-09-08   508.690002   518.349976   508.510010   517.539978   
...          ...          ...          ...          ...          ...   
1254  2020-08-25  3294.989990  3357.399902  3267.000000  3346.489990   
1255  2020-08-26  3351.110107  3451.739990  3344.570068  3441.850098   
1256  2020-08-27  3450.050049  3453.000000  3378.000000  3400.000000   
1257  2020-08-28  3423.000000  3433.370117  3386.500000  3401.800049   
1258  2020-08-31  3408.989990  3495.000000  3405.000000  3450.959961   

        Adj Close   Volume  
0      496.540009  3864500  
1      510.549988  3707100  
2      504.72000

## 하루 주가의 대표값 설정

In [3]:
high_prices = data['High'].values
low_prices = data['Low'].values
mid_prices = (high_prices + low_prices)/2

print(mid_prices.shape)

(1259,)


## sequency data propcessing

In [4]:
import numpy as np

seq_len = 50

def generateX(a, n):
    x_train = []
    y_train = []
    for i in range(len(a)):
        x = a[i:(i + n)]
        if (i + n) < len(a):
            x_train.append(x)
            y_train.append(a[i + n])
        else:
            break
    return np.array(x_train), np.array(y_train)


# Sine 함수에 노이즈를 섞은 데이터로 학습 데이터 100개를 생성한다

x, y = generateX(mid_prices, seq_len)
x = x.reshape(-1,seq_len,1)
y = y.reshape(-1,1)

In [5]:
train_num = 1000
test_num = x.shape[0]-train_num

x_train = x[:train_num, :, :]
y_train = y[:train_num:, :]
x_test = x[train_num:, :, :]
y_test = y[train_num:, :]

## model build

In [8]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, LSTM, GRU, Conv1D
import matplotlib.pyplot as plt

# RNN 모델을 생성 및 학습

xInput = Input(batch_shape=(None, x_train.shape[1], x_train.shape[2]))
x = LSTM(64, return_sequences = True)(xInput)
x = LSTM(64)(x)
x = Dense(64,activation = 'relu')(x)
x = Dense(64,activation = 'relu')(x)
xOutput = Dense(1, activation = 'linear')(x)


model = Model(xInput, xOutput)
model.compile(loss='mse', optimizer='adam')


# 학습
model.fit(x_train, y_train, epochs=500, batch_size=train_num,verbose=1)


 # 예측
y_hat = model.predict(x_test, batch_size=1)

Epoch 1/500
1000/1000 [==============================] - 1s 1ms/sample - loss: 1743012.6250
Epoch 2/500
1000/1000 [==============================] - 0s 148us/sample - loss: 1742676.5000
Epoch 3/500
1000/1000 [==============================] - 0s 122us/sample - loss: 1742447.0000
Epoch 4/500
1000/1000 [==============================] - 0s 119us/sample - loss: 1742240.8750
Epoch 5/500
1000/1000 [==============================] - 0s 123us/sample - loss: 1742036.5000
Epoch 6/500
1000/1000 [==============================] - 0s 120us/sample - loss: 1741815.6250
Epoch 7/500
1000/1000 [==============================] - 0s 126us/sample - loss: 1741580.5000
Epoch 8/500
1000/1000 [==============================] - 0s 138us/sample - loss: 1741360.2500
Epoch 9/500
1000/1000 [==============================] - 0s 134us/sample - loss: 1741112.5000
Epoch 10/500
1000/1000 [==============================] - 0s 123us/sample - loss: 1740829.5000
Epoch 11/500
1000/1000 [==============================] - 0s 

Epoch 88/500
1000/1000 [==============================] - 0s 136us/sample - loss: 1372497.6250
Epoch 89/500
1000/1000 [==============================] - 0s 131us/sample - loss: 1360637.6250
Epoch 90/500
1000/1000 [==============================] - 0s 140us/sample - loss: 1348563.7500
Epoch 91/500
1000/1000 [==============================] - 0s 128us/sample - loss: 1336274.1250
Epoch 92/500
1000/1000 [==============================] - 0s 137us/sample - loss: 1323770.7500
Epoch 93/500
1000/1000 [==============================] - 0s 140us/sample - loss: 1311055.8750
Epoch 94/500
1000/1000 [==============================] - 0s 140us/sample - loss: 1298132.2500
Epoch 95/500
1000/1000 [==============================] - 0s 138us/sample - loss: 1285002.6250
Epoch 96/500
1000/1000 [==============================] - 0s 132us/sample - loss: 1271670.3750
Epoch 97/500
1000/1000 [==============================] - 0s 131us/sample - loss: 1258138.5000
Epoch 98/500
1000/1000 [==========================

1000/1000 [==============================] - 0s 127us/sample - loss: 258839.1406
Epoch 175/500
1000/1000 [==============================] - 0s 138us/sample - loss: 255285.6875
Epoch 176/500
1000/1000 [==============================] - 0s 131us/sample - loss: 252008.9844
Epoch 177/500
1000/1000 [==============================] - 0s 135us/sample - loss: 248997.3281
Epoch 178/500
1000/1000 [==============================] - 0s 141us/sample - loss: 246238.7500
Epoch 179/500
1000/1000 [==============================] - 0s 128us/sample - loss: 243721.0469
Epoch 180/500
1000/1000 [==============================] - 0s 122us/sample - loss: 241431.5156
Epoch 181/500
1000/1000 [==============================] - 0s 127us/sample - loss: 239357.6406
Epoch 182/500
1000/1000 [==============================] - 0s 126us/sample - loss: 237486.7500
Epoch 183/500
1000/1000 [==============================] - 0s 126us/sample - loss: 235805.9844
Epoch 184/500
1000/1000 [==============================] - 0s 13

Epoch 261/500
1000/1000 [==============================] - 0s 129us/sample - loss: 225182.2500
Epoch 262/500
1000/1000 [==============================] - 0s 133us/sample - loss: 225182.2812
Epoch 263/500
1000/1000 [==============================] - 0s 128us/sample - loss: 225182.3594
Epoch 264/500
1000/1000 [==============================] - 0s 132us/sample - loss: 225182.3906
Epoch 265/500
1000/1000 [==============================] - 0s 123us/sample - loss: 225182.4375
Epoch 266/500
1000/1000 [==============================] - 0s 128us/sample - loss: 225182.5000
Epoch 267/500
1000/1000 [==============================] - 0s 130us/sample - loss: 225182.5625
Epoch 268/500
1000/1000 [==============================] - 0s 126us/sample - loss: 225182.6250
Epoch 269/500
1000/1000 [==============================] - 0s 124us/sample - loss: 225182.6406
Epoch 270/500
1000/1000 [==============================] - 0s 132us/sample - loss: 225182.6875
Epoch 271/500
1000/1000 [=========================

Epoch 348/500
1000/1000 [==============================] - 0s 116us/sample - loss: 225182.2500
Epoch 349/500
1000/1000 [==============================] - 0s 119us/sample - loss: 225182.2656
Epoch 350/500
1000/1000 [==============================] - 0s 120us/sample - loss: 225182.2344
Epoch 351/500
1000/1000 [==============================] - 0s 118us/sample - loss: 225182.2344
Epoch 352/500
1000/1000 [==============================] - 0s 112us/sample - loss: 225182.2500
Epoch 353/500
1000/1000 [==============================] - 0s 118us/sample - loss: 225182.2344
Epoch 354/500
1000/1000 [==============================] - 0s 119us/sample - loss: 225182.2344
Epoch 355/500
1000/1000 [==============================] - 0s 115us/sample - loss: 225182.2500
Epoch 356/500
1000/1000 [==============================] - 0s 121us/sample - loss: 225182.2500
Epoch 357/500
1000/1000 [==============================] - 0s 121us/sample - loss: 225182.2344
Epoch 358/500
1000/1000 [=========================

Epoch 435/500
1000/1000 [==============================] - 0s 134us/sample - loss: 225182.2344
Epoch 436/500
1000/1000 [==============================] - 0s 129us/sample - loss: 225182.2500
Epoch 437/500
1000/1000 [==============================] - 0s 127us/sample - loss: 225182.2188
Epoch 438/500
1000/1000 [==============================] - 0s 127us/sample - loss: 225182.2344
Epoch 439/500
1000/1000 [==============================] - 0s 130us/sample - loss: 225182.2500
Epoch 440/500
1000/1000 [==============================] - 0s 124us/sample - loss: 225182.2500
Epoch 441/500
1000/1000 [==============================] - 0s 128us/sample - loss: 225182.2344
Epoch 442/500
1000/1000 [==============================] - 0s 127us/sample - loss: 225182.2344
Epoch 443/500
1000/1000 [==============================] - 0s 124us/sample - loss: 225182.2344
Epoch 444/500
1000/1000 [==============================] - 0s 129us/sample - loss: 225182.2500
Epoch 445/500
1000/1000 [=========================

## 예측 결과 시각화

In [ ]:
y_hat = model.predict(x_test, batch_size=1)



a_axis = np.arange(0, len(y_train))
b_axis = np.arange(len(y_train), len(y_train) + len(y_hat))

plt.figure(figsize=(10,6))
plt.plot(a_axis, y_train.reshape(x_train,), '-')
plt.plot(b_axis, y_hat.reshape(test_num,), '-', color='red', label='Predicted')
plt.plot(b_axis, y_test.reshape(test_num,), '-', color='green', alpha=0.2, label='Actual')
plt.legend()
plt.show()